In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import style
import tables
import copy
import yaml

# want an E^-2 flux
def power_law_flux(e_in_gev):
    return 1E-8 * (e_in_gev**-2.)

### Harvest weights from numpy file

In [ ]:
# load up saved onweights
data = np.load('oneweight_information.npz')

### Event rate in true neutrino energy

In [ ]:
# construct the CDF (which we need for resampling)
hist, bins = np.histogram(data['energy_bin_centers'],bins=data['energy_bins'],weights=data['ehe_energy_weights'])
bin_midpoints = bins[:-1] + np.diff(bins)/2
cdf = np.cumsum(hist)
cdf = cdf/cdf[-1]

def pick_random(cdf, bin_midpoints, num_needed):
        values = np.random.rand(num_needed)
        value_bins = np.searchsorted(cdf, values)
        return bin_midpoints[value_bins]

In [ ]:
# resample the CDF
random_from_cdf = pick_random(cdf, bin_midpoints, 10000)

# plot original data along with resampled data to convince ourselves we did it right
fig2, ax2 = plt.subplots(1, 1 , figsize=(7,5))
ax2.hist(data['energy_bin_centers'], bins=data['energy_bins'],
        weights=data['ehe_energy_weights'], histtype='step', linewidth=3,
        )
ax2.hist(random_from_cdf, bins=data['energy_bins'], histtype='step', linewidth=3)
ax2.set_ylabel("Rate-ish")
ax2.set_xlabel(r'Neutrino Energy [GeV]')
ax2.set_xscale('log')
ax2.set_yscale('log')

## Build the "one weight"

In [ ]:
propagated_weights = data['propagated_weights']
true_e = pick_random(cdf, bin_midpoints, len(propagated_weights))
one_weights = propagated_weights/power_law_flux(true_e)